# Configurações Possíveis no Flask

| [Anterior](8.Templates-parte2.ipynb)| [Próximo](10.Formularios.ipynb)     | 
| :------------- | :----------:|

### 1. Depurando o código no Pycharm

- E como obter o valor dos campos do formulário?
	- No "pycharm", faça um debug e identifique a variável que armazena essas informações.
	- É possível observar que "request.form" é um array que armazena os valores dos campos do formulário HTML.
	- E,por exemplo, o comando "request.form['name']" obtém o valor existente no campo nome do formulário.
	- E, se for necessário transformar os valores do formulário em "json", faça o seguinte:
		1. from json import dumps
		2. return dumps(request.form)
		
- Como depurar uma aplicação com Pycharm?
	- Clique na linha desejada (deixe-a marcada com o ponto vermelho nela)
	- Clique no código + botão direito + debug ou "Run Debug..." ou "Shift + F9"
	- Execute as ações no código HTML
	- Volte para o Pycharm e observe a depuração:
		1. Passeo o mouse sobre a variável desejada (Ex: request) que irá aparecer um menu suspenso
		2. Clique no menu e observe todas as variáveis e valores disponíveis para a variável escolhida



### 2. Arquivos Estáticos

Por padrão, o Flask utiliza o diretório "/static" para guardar arquivos css, javascript e imagens. <br/>
> static/css <br/>
> static/img <br/>
> static/js <br/>

Também é possível armazenar páginas HTMLs estáticas e chamá-la diretamente, sem a necessidade de criar rotas específicas.
> Crie o arquivo 'static_page.html' dentro do diretório '/static'. <br/>
> Acesse "http://127.0.0.1:5000/static/static_page.html" para visualizar o conteúdo do arquivo criado.

- Se for necessário alterar o diretório dos arquivos estáticos:<br/>
> `app = Flask(name, static_folder='public')`

Para este caso, o diretório "public" precisa estar criado.

- Carregando arquivos locais (imagens, css ou javascript)

> < script type="text/javascript" src="{{ url_for('static', filename='js/jquery.js') }}" />

> < img src="{{ url_for('static', filename='img/home.png') }}" />

### 3. Macros (Poderia fazer com `Macros` algo semelhante a `Widgets`?)

Quando se deseja reduzir duplicação de código:

> `_macro.html`

```
{% macro show_list(value) %}
    <h1> {{ value }} </h1>
    <h2> {{ value }} </h2>
    <h3> {{ value }} </h3>
{% endmacro %}
```

> `index.html`

```
{% extends 'base/base.html' %}
{% block title %} {{ title }} {% endblock %}

{% block content %} 
    {% from "_macro.html" import show_list %}
    {{ show_list( title ) }} 
{% endblock %}
```

### 4. Error 404

Uma forma de tratar as chamadas a URLs não existentes.

1. Para começar, o decorator utilizado será o `errohandler` na página principal (controller) da aplicação `app.py`

```
from flask import render_template

@app.errorhandler(403)
@app.errorhandler(404)
def page_not_founde(e):
    return render_template('404.html')
```    

2. Crie a página 404.html

```
<html>
    ...
    <body>
        <p class="message_error">Desculpem-nos pelos transtornos! Esta página não foi encontrada.</p>
    </body>
    ...
</html>
```

### 5. `Before` e `After` requests (`Decorators`)

Um exemplo de uma situação comum desse tipo de `Decorator` é quando necessitamos realizar uma consulta a uma banco de dados. Previamente é necessário saber se uma conexão com o banco de dados já foi criada.

- **Decorador `before_request`**

```
Realiza uma tarefa antes de iniciar a requisição.

from flask import request

@app.before_request()
def before_request():
    if 'username' not in session:
        print 'Você está na página {}'.format(request.endpoint)
        print 'O usuário não fez login.'
```

- **Decorador `after_request`**

```
Realiza uma tarefa logo após de uma requisição.
Sempre deverá retornar o response do parâmetro da função.
Um exemploque poder ser útil é o de finalizar uma conexão com o banco de dados após realizar uma consulta.

from flask import request

@app.after_request()
def after_request(response):
   print "Do anything" 
   return response
```

### 6. Global

Variáveis que serão compartilhadas por toda a aplicação. <br/>
Variáveis globais ainda não são thread-safe porque ainda não há proteção contra a maioria das condições de corrida. <br/>
Para trabalhar com variáveis globais...

* **1. Importe a biblioteca**
> `from flask import g` 

* **2. Crie uma variável global**

```
@app.before_request()
def before_request():
    g.id_number = 1
```

* **3. Obtenha o valor de uma variável global**

```
@app.route("/index")
def index():
    print g.id_number
    ...
    return ...
```

### 7. Config

* **1. Criando arquivos de configuração para ambientes diferente**

1. Crie o arquivo `config.py`
2. Crie a classe Config:

```
class Config(object)
    SECRET_KEY = 'jshqw7723uy32iqwk0912'

class DevelopmentConfig(Config):
    DEBUG=True
    PORT=8000
    
class ProductionConfig(Config):
    DEBUG=False 
    PORT=8181
```

* **3. Import as configurações no seu controlador principal `app.py`**

```
from config import DevelopmentConfig
...

app = Flask(__ name __)
app.config.from_object(DevelopmentConfig)
...

if __ name __ == '__ main __':
    app.run(port=config.PORT)
```